In [1]:
from pathlib import Path
from openai import OpenAI
import os
import pandas as pd
import nltk
import ast
from collections import Counter
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [47]:
#Load API key
my_key = open(R"C:\\Users\\pauld\\OneDrive\\PhD\\Code\\keys\\open_ai.txt", "r").read()

In [48]:
client = OpenAI(api_key=my_key)

In [56]:
# open patent data 
df = pd.read_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked_fulltxts_sentences_GPT_pve.csv', index_col = 0) 

In [58]:
df

,keyword,patent_id,sentence,GPT response
0,hypertriglyceridemia,US10227392B2,"In one embodiment, the treated or prevented di...",Direct PVE; The sentence explicitly mentions t...
1,nutraceutical,US10829778B2,HPPD-inhibiting herbicides-tolerant plants her...,Direct PVE; The sentence explicitly mentions t...
2,environmental pressure,AU2017235608B2,"As it appears from the Examples herein, four l...",No PVE; The sentence describes technical proce...
3,environmentally friendl,US20140024064A1,Production of alcohols by fermentation is one ...,Direct PVE; This sentence directly mentions an...
4,resistan,US9534232B2,"Furthermore, the continual use of insecticides...",Direct PVE; The sentence highlights the advers...
...,...,...,...,...
128,restoration,US7892824B2,Dystrophin expression and nNOS restoration wer...,No PVE; The sentence focuses on the examinatio...
129,public,US8859237B2,It is preferred to use the public default valu...,No PVE; The sentence is focused on technical i...
130,extinction,US6489155B1,The bonds thus cleaved are determined by takin...,No PVE; The sentence describes a technical pro...
131,bioremediation,US7183105B2,"Undesirable substances include, but are not li...",Contextual PVE; This sentence indirectly impli...


In [92]:
df

,keyword,patent_id,sentence,GPT response
0,hypertriglyceridemia,US10227392B2,"In one embodiment, the treated or prevented di...",Direct PVE; The sentence explicitly mentions t...
1,nutraceutical,US10829778B2,HPPD-inhibiting herbicides-tolerant plants her...,Direct PVE; The sentence explicitly mentions t...
2,environmental pressure,AU2017235608B2,"As it appears from the Examples herein, four l...",No PVE; The sentence describes technical proce...
3,environmentally friendl,US20140024064A1,Production of alcohols by fermentation is one ...,Direct PVE; This sentence directly mentions an...
4,resistan,US9534232B2,"Furthermore, the continual use of insecticides...",Direct PVE; The sentence highlights the advers...
...,...,...,...,...
128,restoration,US7892824B2,Dystrophin expression and nNOS restoration wer...,No PVE; The sentence focuses on the examinatio...
129,public,US8859237B2,It is preferred to use the public default valu...,No PVE; The sentence is focused on technical i...
130,extinction,US6489155B1,The bonds thus cleaved are determined by takin...,No PVE; The sentence describes a technical pro...
131,bioremediation,US7183105B2,"Undesirable substances include, but are not li...",Contextual PVE; This sentence indirectly impli...


In [61]:
#GPT
nr = 0
patent_ids = []
sentences = []
gpt_responses_ind = []
keywords = []
gpt_response_pve = []
for sentence in list(df['sentence']):
            my_chat_bot = "You are a helpful assistant that links mentioned technologies, genes, genomes, enzymes, or metabolic pathways to actual or potential industrial applications. Examples for industrial applications can be food products, biofuels, fertilizers, detergents, medicines, cosmetics, seeds, etc. They have to be more concrete then just 'biotechnology' or 'enzyme'. Also look for other types of applications that are not amongst the mentioned examples. Do not under any circumstances base your label on applications mentioned in the sentence. Just identify the mentioned technology/gene/etc. and judge on your own terms whether they are likely to be useful for any applications. For each sentence, return a label indicating either the types of industrial application (formulated in maximum three words per application) or 'no appl'. After the label, provide an explanation for your labelling choice in a maximum of two sentences. Separate the label and explanation by ';'."      
            query = f"Please link any technologies, genes, genomes, enzymes, or metabolic pathways mentioned in the following sentence to actualy or potential industrial applications? The sentence is: {sentence[:4096]}"
            
            response = client.chat.completions.create(
                model="gpt-4-turbo-preview",
                messages = [{"role": "system", "content": my_chat_bot},
                            {"role": "user", "content": query}],
            max_tokens=200,
            )
            keywords.append(list(df['keyword'])[nr])
            patent_ids.append(list(df['patent_id'])[nr])
            sentences.append(list(df['sentence'])[nr])
            gpt_response_pve.append(list(df['GPT response'])[nr])
            gpt_responses_ind.append(response.choices[0].message.content)
            nr = nr+1
            

In [93]:
df_analysed = pd.DataFrame(zip(list(df['keyword']), patent_ids, sentences, gpt_response_pve, gpt_responses_ind), columns = ['keyword','patent_id', 'sentence', 'GPT response_pve', 'GPT response_ind'])

In [85]:
# get another sample of 13 sentences to evaluate the labelling decisions
df_analysed_sample = df_analysed.sample(n=26, random_state = 3).reset_index(drop = True)


In [89]:
nr = 0
for row in df_analysed_sample.index: 
    print(list(df_analysed_sample['sentence'])[nr])
    print(list(df_analysed_sample['GPT response_ind'])[nr])
    print('-------------\n')
    nr = nr+1
    
    

Several bacterial strains belonging to the genus Persephonella have been isolated from deep-sea hydrothermal vents.
Bioremediation, Biofuels; Bacteria from extreme environments like deep-sea hydrothermal vents often have unique enzymes and metabolic pathways that can be exploited industrially for the degradation of pollutants (bioremediation) or the production of biofuels due to their ability to function under extreme conditions.
-------------

Proline A and Proline B together include 8.8 mM NaNO3, 0.361 mM NaH2PO4.H2O, 10×F/2 Trace metals, and 10×F/2 Vitamins (Guillard (1975) Culture of phytoplankton for feeding marine invertebrates.
no appl; The sentence describes specific concentrations of chemical compounds and nutrients in a culture medium designed for the growth of phytoplankton, rather than mentioning specific technologies, genes, genomes, enzymes, or metabolic pathways that would directly imply industrial applications.
-------------

The term “biological property” of the protei

In [95]:
df_analysed.to_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked_fulltxts_sentences_GPT_pve_ind.csv')

In [67]:
# get another sample of 13 sentences to evaluate the labelling decisions
df_analysed_sample = df_analysed.sample(n=26, random_state = 3)


In [68]:
df_analysed_sample

,patent_id,sentence,GPT response
25,US7517678B2,The hyperthermophile Aeropyrum pernix is an ae...,No PVE; This sentence provides background info...
122,AU2005217079B2,Culture of phytoplankton for feeding marine in...,Direct PVE; The sentence explicitly mentions t...
114,WO2009153208A1,Then add:Deionized water 0.10Reagent J (Enzyme...,No PVE; The sentence only includes technical i...
6,AU2008209677B2,15 Identified by their high degree of conserva...,No PVE; The sentence discusses the technical a...
126,EP3332028B1,"In Europe, 2.5 million extra hospital days lea...",Contextual PVE; The sentence mentions the soci...
98,US8207363B2,"In some embodiments, the culture temperature i...",No PVE; The sentence only focuses on technical...
34,US10704050B2,More specifically this invention relates to a ...,No PVE; The sentence refers to technical aspec...
67,US10570407B2,A selection marker using the D-form of amino a...,No PVE; The sentence describes technical aspec...
35,US7393923B2,"The largest wavelength of excitation (598 nm),...",No PVE; The sentence provides technical inform...
8,US6140486A,Sources of ORFs having PKS-like genes are thos...,No PVE; The sentence mentions the characterist...


In [69]:
print_full(df_analysed_sample)

          patent_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [82]:
# repeat procedure with adjusted prompt
#GPT
nr = 0
patent_ids_2 = []
sentences_2 = []
responses_2 = []
keywords_2 = []
for sentence in list(df_kw_finalsample['sentence']):
            my_chat_bot = "You are a helpful assistant that categorizes the sentences into 'Public Value Expression' (PVE) or 'No PVE'. If the sentence is categorized as ‘PVE’, further classify it into Direct PVE or ‘Contextual PVE’. Use the following definitions: Public value expression (PVE): A public value expression is one that indicates societal benefits promised to or for people or ecosystems. Societal benefits for people includes, among other aspects, potential enhancement of safety, security, justice, equality, human rights, and societal or global health. Societal benefits for ecosystems include, among other aspects, enhancements to sustainability, nature and the environment. Enhancement includes both positive improvement and the mitigation or prevention of adverse conditions. Public value is based on the creation of shared utility for users, people, or nature. No Public Value: Sentences without any explicit indication of societal benefits beyond are the opposite of PVE. This may be the case if they do only explain some technological process without reference to any PVE. Furthermore, if sentences describe benefits of a technology in a manner that does not explicitly link to public values, it should not be considered PVE. Furthermore, do not code a sentence as 'PVE' if it only refers to improving individual user experience, value, health. Explanations of how an invention changes environmental properties, without addressing societal or environmental public value do not imply public value expression. Avoid assumptions about possible positive or negative societal, economic, or environmental spillovers if they are not explicitly stated. Direct PVE: Code a sentence as a 'Direct PVE' if the sentence states that the invention being patented explicitly addresses one or more PVEs (see definition of PVE). Contextual PVE: Code a sentence as a 'Contextual PVE' if the sentence mentions a contextual situation that allows us to infer a concern or intention to create or protect PVEs. The sentence must be explicit in its reference to a public value or concern about a societal, environmental, or ethical concern. However, it is contextual because it does not explicitly connect the invention to the public value. Further instructions: The format of your response should be the following: first, assign one of the labels ('No PV'’,  'Direct PVE', or 'Contextual PVE'). Second, add one or two sentences of easy-to-understand explanation on why you assigned the specific label. The explanation must not be longer than two sentences maximum. Always separate label and explanation by ';'."
            example_query_1 = "Does the following sentence include a 'Direct PVE', a 'Contextual PVE', or 'No PVE'? The sentence is: 'These properties allow for moisture and/or nutrients dissolved therein to be retained and/or transported by the solid support so that they are available to the microorganisms growing on the surface.The properties of the cultivation support, especially moisture and/or nutrient retention, can be enhanced by coating the support with a material selected to enhance photosynthetic microorganism growth."                                                                                                                           
            example_response_1 = "No PVE; The sentence explains how the invention supports the growth of photosynthetic microorganisms, without explicitly mentioning public value related to such growth. No public value is explicitly stated."  
            example_query_2 = "Does the following sentence include a 'Direct PVE', a 'Contextual PVE', or 'No PVE'? The sentence is: 'A selection marker using the D-form of amino acids and based on the fact that plants can only tolerate the L-form offers a fast, efficient and environmentally friendly selection system. Subsequently, a plant may be regenerated, eg from single cells, callus tissue or leaf discs, as is standard in the art.'"                                                                                                                           
            example_response_2 = "Direct PVE; It describes a concrete biotechnological selection system, explicitly linking it to efficient and environmentally friendly qualities. Environmentally friendly qualities are per definition considered to constitute public value, and, if directly linked to the patented invention, should be classified as direct PVE.'"  
            example_query_3 = "Does the following sentence include a 'Direct PVE', a 'Contextual PVE', or 'No PVE'? The sentence is: 'Because of this lack of knowledge about the mechanism of CK2 activation and its constitutive activity, CK2 has been mostly neglected as a key target for anti-cancer drugs.'"                                                                    
            example_response_3 = "Contextual PVE; The sentence implies a concern about the neglect of CK2 as a target for anti-cancer drugs, indicating a broader societal value in finding potential treatments for cancer. However, it is not directly connected to the invention being patented.'"
            query = f"Does the following sentence include a 'Direct PVE', a 'Contextual PVE', or 'No PVE'? The sentence is: {sentence[:4096]}"
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages = [{"role": "system", "content": my_chat_bot},
                            {"role": "user", "content": example_query_1},
                            {"role": "assistant", "content": example_response_1},
                            {"role": "user", "content": example_query_2},
                            {"role": "assistant", "content": example_response_2},
                            {"role": "user", "content": example_query_3},
                            {"role": "assistant", "content": example_response_3}, 
                            {"role": "user", "content": query}],
            max_tokens=200,
            )
            keywords_2.append(list(df_kw_finalsample.index)[nr])
            patent_ids_2.append(list(df_kw_finalsample['patent_id'])[nr])
            sentences_2.append(list(df_kw_finalsample['sentence'])[nr])
            responses_2.append(response.choices[0].message.content)
            nr = nr+1
            

df_analysed_2 = pd.DataFrame(zip(keywords_2, patent_ids_2, sentences_2, responses_2), columns = ['keyword','patent_id', 'sentence', 'GPT response'])

In [91]:
# get same sample of 26 sentences to evaluate the labelling decisions after the updated prompt
df_analysed_sample_2 = df_analysed_2.sample(n=26, random_state = 3)
df_analysed_sample_2

,keyword,patent_id,sentence,GPT response
25,hydrothermal vent,US7517678B2,The hyperthermophile Aeropyrum pernix is an ae...,No PVE; The sentence provides factual informat...
122,phytoplankton,AU2005217079B2,Culture of phytoplankton for feeding marine in...,Contextual PVE; The sentence implies a context...
114,extinction,WO2009153208A1,Then add:Deionized water 0.10Reagent J (Enzyme...,No PVE; The sentence provides instructions for...
6,conservation,AU2008209677B2,15 Identified by their high degree of conserva...,No PVE; The sentence explains the use of a met...
126,societal,EP3332028B1,"In Europe, 2.5 million extra hospital days lea...",Contextual PVE; The sentence mentions the soci...
98,omega-3,US8207363B2,"In some embodiments, the culture temperature i...",No PVE; The sentence provides technical detail...
34,society,US10704050B2,More specifically this invention relates to a ...,Contextual PVE; The sentence provides backgrou...
67,environmentally friendl,US10570407B2,A selection marker using the D-form of amino a...,Direct PVE; The sentence explicitly states tha...
35,emission,US7393923B2,"The largest wavelength of excitation (598 nm),...",No PVE; The sentence provides information abou...
8,deep sea,US6140486A,Sources of ORFs having PKS-like genes are thos...,Contextual PVE; The sentence mentions the prod...


In [85]:
print_full(df_analysed_sample_2)

          patent_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [92]:
df_analysed_2.to_csv(R'X:\5_Research\Paul\data\genes_of_technology\full_patent_data_lens_marine_2689_familychecked_fulltxts_sentences_GPT.csv')